# How-to deploy a model on Azure ML

This notebook takes you through the steps of deploying a model on Azure for The Ocean Cleanup. We can deploy any model registered after the training as seen in the notebooks of the `ModelTraining` directory.

Next to the concepts introduced in those notebooks, there are a few additional concepts we need to know about:

- score script: A Python script that contains the code of how to perform inference. Implements two functions:
  - `init()`: Initializing function run once when the container is started. Initialize your model here
  - `run(request)`: Function that is called for every request to the endpoint. This endpoint is called with the
    binary image data as body, which is provided to the function through request.get_data(). The response is
    sent back.
- Service: A deployed model. This provides an `endpoint` to which we can send data.
- Endpoint: This is provided by a service. Is an URI to which inference requests can be sent.
- InferenceConfig: Configuration describing how the inference should be performed. This contains the score script,
  the source it is run in and the environment it should use.
- DeployConfig: The configuration of where to deploy the service. This can be a LocalWebservice, AciWebservice or
  AksWebservice. The former 2 are aimed at development work, the later for production.
- Compute Target: When using an AksWebservice, the Compute Target must be named. This is the AKS cluster where to
  run the service. This should be registered as 'Inference cluster' in Azure ML.
  
Before we get to work with these, we need to load in our workspace again, like before:

In [3]:
from toc_azurewrapper.workspace import get_workspace

subscription_id = "a00eaec6-b320-4e7c-ae61-60a30aec1cfc"
resource_group = "MachineLearning"
workspace_name = "RiverImageAnalysis"
tenant_id = "86f9fea7-9eb0-4325-8b58-7ed0db623956"

workspace = get_workspace(subscription_id, resource_group, workspace_name, tenant_id=tenant_id)

Now, like before, we need to create or load an environment:

In [2]:
from toc_azurewrapper.environment import get_environment

environment = get_environment(
    workspace,
    "tensorflow-yolo",
    pip_requirements="./examples/yolo/Tensorflow_YOLO/requirements.txt",
    docker_image="mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04"
)

NameError: name 'workspace' is not defined

Now it's time to load the model. Note that it is possible to provide multiple models to the Service. However, in the current architectural design, we deploy a single model as a single service, and therefore single endpoint.

In [7]:
from azureml.core import Model

model = Model(workspace, "yolo_v4")

No environment with that name found, creating new one


At this time, the score script should be implemented. The score script implements an `init()` function, and a `run(request)` function. Note that we will deploy the entire folder containing the `score.py` script, so it is perfectly fine to use imports within that and spread your code over multiple files.

The provided model is used as follows: The artifacts stored with the model will be loaded into a folder, that is pointed to by the environment variable `AZUREML_MODEL_DIR`. When multiple models are provided, there will be an extra layer of subdirectories there, where each folder has the name of a model used. The artifacts will then be inside that folder.

The folder structure is kept the same as shown in the `Artifacts` tab of the Azure ML Studio interface for a model.

## Deploying the model

Once we have the score script ready, it's time to deploy the model. The function `azurewrapper.deploy.deploy()` can help with this. It will create the correct InferenceConfig and DeployConfig, and will use those to create the service.

In [12]:
from toc_azurewrapper.deploy import deploy
service = deploy(
    workspace,
    "yolo-v4",
    model,
    "score.py",
    "examples/yolo",
    environment=environment,
    target="aks",
    compute_target_name="inference-ML"
)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................................................................
Succeeded
AKS service creation operation finished, operation "Succeeded"


In [ ]:
service.scoring_uri